In [1]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [2]:
import re

rows = []

with open("data.txt", "r") as f:
    for row in f:
        row = re.sub(r'\([^)]*\)', '', row)
        for char in row:
            if char == '[':
                row = row.replace('[', '')
            elif char == ']':
                row = row.replace(']', '')
        row= re.sub(r'\s+', ' ', row)
        rows.append(row)

with open("clean_data.txt", "w") as f:
    for row in rows:
        f.write(row + '\n')   

In [7]:
import os

aug = naw.SynonymAug(aug_src='wordnet')
augmented_text = aug.augment(rows)
with open("aug_data.txt", "w") as f:
    for text in augmented_text:
        f.write(text+ '\n')

In [15]:
from googletrans import Translator

def backtranslate(sentence):
    translator = Translator()
    # Translate the sentence to a foreign language
    translation = translator.translate(sentence, dest='fr')
    # Translate the translation back to the original language
    back_translation = translator.translate(translation.text, dest='en')
    # Return the back-translated text
    return back_translation.text

aug_back_rows = []
for sentence in rows:
    aug_back_rows.append(backtranslate(sentence))

with open("aug_back_data.txt", "w") as f:
    for text in aug_back_rows:
        f.write(text+ '\n')

In [26]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the files
with open('clean_data.txt', 'r') as file:
    rows = [line.strip() for line in file]

with open('aug_data.txt', 'r') as file:
    rows1 = [line.strip() for line in file]
    
with open('aug_back_data.txt', 'r') as file:
    rows2 = [line.strip() for line in file]

# Preprocess the rows
tokenizer = nltk.RegexpTokenizer(r'\w+')
stop_words = nltk.corpus.stopwords.words('english')
stemmer = nltk.PorterStemmer()

def preprocess(text):
    tokens = tokenizer.tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

rows1 = [preprocess(row) for row in rows1]
rows2 = [preprocess(row) for row in rows2]

# Create a document-term matrix
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(rows)
matrix1 = vectorizer.transform(rows1)
matrix2 = vectorizer.transform(rows2)

# Compute similarity
similarity1 = np.mean(np.diag(np.array(cosine_similarity(matrix, matrix1))))
similarity2 = np.mean(np.diag(np.array(cosine_similarity(matrix, matrix2))))
print(similarity1)
print(similarity2)

0.5839810003127641
0.568985023266408
